In [1]:
import pandas as pd
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 10000)
import numpy as np
import matplotlib.pyplot as plt
import math
import warnings
warnings.filterwarnings("ignore")

# TensorFlow 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.nn import relu

# Scikit Learn 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor, ElasticNet, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
with open('data_description.txt', 'r') as text_file:
    
    text_features_dict = {
        'Id':[np.nan], 
        'SalePrice':[np.nan]
    }
    
    for line in text_file:
        if ':' in line and line[0] != ' ':
            text_column = line[:line.find(":")]
            if (text_column == 'Bedroom' or text_column == 'Kitchen'):
                text_column += 'AbvGr'
            text_features_dict[text_column] = [np.nan]
        elif line.strip() != '':
            text_value = line[:line.find("\t")].strip()
            text_features_dict[text_column].append(text_value)
            if np.nan in text_features_dict[text_column]:
                text_features_dict[text_column].remove(np.nan) 

del text_file, line, text_column, text_value

In [3]:
text_features_dict

{'Id': [nan],
 'SalePrice': [nan],
 'MSSubClass': ['20',
  '30',
  '40',
  '45',
  '50',
  '60',
  '70',
  '75',
  '80',
  '85',
  '90',
  '120',
  '150',
  '160',
  '180',
  '190'],
 'MSZoning': ['A', 'C', 'FV', 'I', 'RH', 'RL', 'RP', 'RM'],
 'LotFrontage': [nan],
 'LotArea': [nan],
 'Street': ['Grvl', 'Pave'],
 'Alley': ['Grvl', 'Pave', 'NA'],
 'LotShape': ['Reg', 'IR1', 'IR2', 'IR3'],
 'LandContour': ['Lvl', 'Bnk', 'HLS', 'Low'],
 'Utilities': ['AllPub', 'NoSewr', 'NoSeWa', 'ELO'],
 'LotConfig': ['Inside', 'Corner', 'CulDSac', 'FR2', 'FR3'],
 'LandSlope': ['Gtl', 'Mod', 'Sev'],
 'Neighborhood': ['Blmngtn',
  'Blueste',
  'BrDale',
  'BrkSide',
  'ClearCr',
  'CollgCr',
  'Crawfor',
  'Edwards',
  'Gilbert',
  'IDOTRR',
  'MeadowV',
  'Mitchel',
  'Names',
  'NoRidge',
  'NPkVill',
  'NridgHt',
  'NWAmes',
  'OldTown',
  'SWISU',
  'Sawyer',
  'SawyerW',
  'Somerst',
  'StoneBr',
  'Timber',
  'Veenker'],
 'Condition1': ['Artery',
  'Feedr',
  'Norm',
  'RRNn',
  'RRAn',
  'PosN',
  

In [4]:
numerical_features_lst = []
categorical_features_lst  = []
special_cases_features_lst  = []

for key, value in text_features_dict.items():
    if np.nan in value:
        numerical_features_lst.append(key)
    else:
        if 'NA' in value:
            special_cases_features_lst.append(key)
        else:
            categorical_features_lst.append(key)

del key, value
len(numerical_features_lst), len(categorical_features_lst), len(special_cases_features_lst)

(35, 32, 14)

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [6]:
def missing_values(train=train, test=test):
    train_null = train.isnull().sum()
    test_null = test.isnull().sum()
    return pd.DataFrame({
        'Train': train_null, 
        'Test': test_null,
        '% Train': round(train_null/len(train), 2),
        '% Test': round(test_null/len(test), 2)
    })

In [7]:
missing_values(train[special_cases_features_lst], test[special_cases_features_lst]).query("Train > 0 or Test > 0")

,Train,Test,% Train,% Test
Alley,1369,1352,0.94,0.93
BsmtQual,37,44,0.03,0.03
BsmtCond,37,45,0.03,0.03
BsmtExposure,38,44,0.03,0.03
BsmtFinType1,37,42,0.03,0.03
BsmtFinType2,38,42,0.03,0.03
FireplaceQu,690,730,0.47,0.50
GarageType,81,76,0.06,0.05
GarageFinish,81,78,0.06,0.05
GarageQual,81,78,0.06,0.05


In [8]:
# Replace categorical missing values with Not Available  
for column in special_cases_features_lst:
    print(f'\n******* {column} *******')
    print('--- TRAIN ---')
    print('Before:', train[column].unique(), end='\n')
    train[column] = train[column].apply(lambda value: f'No {column}' if pd.isna(value) else value)
    print('After:', train[column].unique(), end='\n\n')
    print('--- TEST ---')
    print('Before:', test[column].unique(), end='\n')
    test[column] = test[column].apply(lambda value: f'No {column}' if pd.isna(value) else value)
    print('After:', test[column].unique(), end='\n\n')

del column


******* Alley *******
--- TRAIN ---
Before: [nan 'Grvl' 'Pave']
After: ['No Alley' 'Grvl' 'Pave']

--- TEST ---
Before: [nan 'Pave' 'Grvl']
After: ['No Alley' 'Pave' 'Grvl']


******* BsmtQual *******
--- TRAIN ---
Before: ['Gd' 'TA' 'Ex' nan 'Fa']
After: ['Gd' 'TA' 'Ex' 'No BsmtQual' 'Fa']

--- TEST ---
Before: ['TA' 'Gd' 'Ex' 'Fa' nan]
After: ['TA' 'Gd' 'Ex' 'Fa' 'No BsmtQual']


******* BsmtCond *******
--- TRAIN ---
Before: ['TA' 'Gd' nan 'Fa' 'Po']
After: ['TA' 'Gd' 'No BsmtCond' 'Fa' 'Po']

--- TEST ---
Before: ['TA' 'Po' 'Fa' 'Gd' nan]
After: ['TA' 'Po' 'Fa' 'Gd' 'No BsmtCond']


******* BsmtExposure *******
--- TRAIN ---
Before: ['No' 'Gd' 'Mn' 'Av' nan]
After: ['No' 'Gd' 'Mn' 'Av' 'No BsmtExposure']

--- TEST ---
Before: ['No' 'Gd' 'Mn' 'Av' nan]
After: ['No' 'Gd' 'Mn' 'Av' 'No BsmtExposure']


******* BsmtFinType1 *******
--- TRAIN ---
Before: ['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' nan 'LwQ']
After: ['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' 'No BsmtFinType1' 'LwQ']

--- TEST ---
Before: ['Rec'

In [9]:
missing_values(train[special_cases_features_lst], test[special_cases_features_lst]).query("Train > 0 or Test > 0")

,Train,Test,% Train,% Test


In [10]:
missing_values(train[categorical_features_lst], test[categorical_features_lst]).query("Train > 0 or Test > 0")

,Train,Test,% Train,% Test
MSZoning,0,4,0.00,0.00
Utilities,0,2,0.00,0.00
Exterior1st,0,1,0.00,0.00
Exterior2nd,0,1,0.00,0.00
MasVnrType,8,16,0.01,0.01
Electrical,1,0,0.00,0.00
KitchenQual,0,1,0.00,0.00
Functional,0,2,0.00,0.00
SaleType,0,1,0.00,0.00


In [11]:
print('******* TRAIN *******')
for column in categorical_features_lst:
    if np.nan in list(train[column].unique()):
        print(f'\n{column}:')
        print('Before:', train[column].unique(), end='\n')
        train[column] = train[column].apply(lambda value: train[column].mode()[0] if pd.isna(value) else value)
        print('After:', train[column].unique(), end='\n')

print('\n******* TEST *******')
for column in categorical_features_lst:
    if np.nan in list(test[column].unique()):
        print(f'\n{column}:')
        print('Before:', test[column].unique(), end='\n')
        test[column] = test[column].apply(lambda value: test[column].mode()[0] if pd.isna(value) else value)
        print('After:', test[column].unique(), end='\n')
        
del column

******* TRAIN *******

MasVnrType:
Before: ['BrkFace' 'None' 'Stone' 'BrkCmn' nan]
After: ['BrkFace' 'None' 'Stone' 'BrkCmn']

Electrical:
Before: ['SBrkr' 'FuseF' 'FuseA' 'FuseP' 'Mix' nan]
After: ['SBrkr' 'FuseF' 'FuseA' 'FuseP' 'Mix']

******* TEST *******

MSZoning:
Before: ['RH' 'RL' 'RM' 'FV' 'C (all)' nan]
After: ['RH' 'RL' 'RM' 'FV' 'C (all)']

Utilities:
Before: ['AllPub' nan]
After: ['AllPub']

Exterior1st:
Before: ['VinylSd' 'Wd Sdng' 'HdBoard' 'Plywood' 'MetalSd' 'CemntBd' 'WdShing'
 'BrkFace' 'AsbShng' 'BrkComm' 'Stucco' 'AsphShn' nan 'CBlock']
After: ['VinylSd' 'Wd Sdng' 'HdBoard' 'Plywood' 'MetalSd' 'CemntBd' 'WdShing'
 'BrkFace' 'AsbShng' 'BrkComm' 'Stucco' 'AsphShn' 'CBlock']

Exterior2nd:
Before: ['VinylSd' 'Wd Sdng' 'HdBoard' 'Plywood' 'MetalSd' 'Brk Cmn' 'CmentBd'
 'ImStucc' 'Wd Shng' 'AsbShng' 'Stucco' 'CBlock' 'BrkFace' 'AsphShn' nan
 'Stone']
After: ['VinylSd' 'Wd Sdng' 'HdBoard' 'Plywood' 'MetalSd' 'Brk Cmn' 'CmentBd'
 'ImStucc' 'Wd Shng' 'AsbShng' 'Stucco' 'CBl

In [12]:
print("\n********* TRAIN *********")
for column in train.columns:
    for value in train[column].unique():
        if value == 'None':
            print(f'\n{column} has None values')
            train[column] = train[column].apply(lambda record: f'No {column}' if record == 'None' else record)
            print('REPLACED:', train[column].unique())
    
print("\n********* TEST  *********")
for column in test.columns:
    for value in test[column].unique():
        if value == 'None':
            print(f'\n{column} has None values')
            test[column] = test[column].apply(lambda record: f'No {column}' if record == 'None' else record)
            print('REPLACED:', test[column].unique())

del column


********* TRAIN *********

MasVnrType has None values
REPLACED: ['BrkFace' 'No MasVnrType' 'Stone' 'BrkCmn']

********* TEST  *********

MasVnrType has None values
REPLACED: ['No MasVnrType' 'BrkFace' 'Stone' 'BrkCmn']


In [13]:
missing_values(train[categorical_features_lst], test[categorical_features_lst]).query("Train > 0 or Test > 0")

,Train,Test,% Train,% Test


In [14]:
missing_values(train[numerical_features_lst[2:]], test[numerical_features_lst[2:]]).query("Train > 0 or Test > 0")

,Train,Test,% Train,% Test
LotFrontage,259,227,0.18,0.16
MasVnrArea,8,15,0.01,0.01
BsmtFinSF1,0,1,0.00,0.00
BsmtFinSF2,0,1,0.00,0.00
BsmtUnfSF,0,1,0.00,0.00
TotalBsmtSF,0,1,0.00,0.00
BsmtFullBath,0,2,0.00,0.00
BsmtHalfBath,0,2,0.00,0.00
GarageYrBlt,81,78,0.06,0.05
GarageCars,0,1,0.00,0.00


In [15]:
def compare_columns(dataset, missing_column, compare_column):
    return dataset[pd.isna(dataset[missing_column])][[missing_column, compare_column]]

In [16]:
compare_columns(train, 'LotFrontage', 'Street')

,LotFrontage,Street
7,NaN,Pave
12,NaN,Pave
14,NaN,Pave
16,NaN,Pave
24,NaN,Pave
31,NaN,Pave
42,NaN,Pave
43,NaN,Pave
50,NaN,Pave
64,NaN,Pave


In [17]:
print('\n*** TRAIN mean ***')
print('LotFrontage:', float((train['LotFrontage'].mean())))
print('LotFrontage (Pave Street):', float(train[train['Street']=='Pave'][['LotFrontage', 'Street']].mean()))
print('\n*** TEST mean ***')
print('LotFrontage:', float((test['LotFrontage'].mean())))
print('LotFrontage (Pave Street):', float(test[test['Street']=='Pave'][['LotFrontage', 'Street']].mean()))


*** TRAIN mean ***
LotFrontage: 70.04995836802665
LotFrontage (Pave Street): 69.98578595317726

*** TEST mean ***
LotFrontage: 68.58035714285714
LotFrontage (Pave Street): 68.48899755501222


In [18]:
train_lotfront_age_mean = float(math.floor(train['LotFrontage'].mean()))
test_lotfront_age_mean = float(math.floor(test['LotFrontage'].mean()))

train['LotFrontage'] = train['LotFrontage'].apply(lambda distance: train_lotfront_age_mean if pd.isna(distance) else distance)
test['LotFrontage'] = test['LotFrontage'].apply(lambda distance: test_lotfront_age_mean if pd.isna(distance) else distance)

del train_lotfront_age_mean, test_lotfront_age_mean

In [19]:
compare_columns(train, 'MasVnrArea', 'MasVnrType')

,MasVnrArea,MasVnrType
234,NaN,No MasVnrType
529,NaN,No MasVnrType
650,NaN,No MasVnrType
936,NaN,No MasVnrType
973,NaN,No MasVnrType
977,NaN,No MasVnrType
1243,NaN,No MasVnrType
1278,NaN,No MasVnrType


In [20]:
train['MasVnrArea'] = train['MasVnrArea'].apply(lambda area: 0.0 if pd.isna(area) else area)
test['MasVnrArea'] = test['MasVnrArea'].apply(lambda area: 0.0 if pd.isna(area) else area)

In [21]:
compare_columns(train, 'GarageYrBlt', 'GarageCond')

,GarageYrBlt,GarageCond
39,NaN,No GarageCond
48,NaN,No GarageCond
78,NaN,No GarageCond
88,NaN,No GarageCond
89,NaN,No GarageCond
99,NaN,No GarageCond
108,NaN,No GarageCond
125,NaN,No GarageCond
127,NaN,No GarageCond
140,NaN,No GarageCond


In [22]:
train['GarageYrBlt'] = train['GarageYrBlt'].apply(lambda year: 0.0 if pd.isna(year) else year)
test['GarageYrBlt'] = test['GarageYrBlt'].apply(lambda year: 0.0 if pd.isna(year) else year)

In [23]:
missing_values(train[numerical_features_lst[2:]], test[numerical_features_lst[2:]]).query("Train > 0 or Test > 0")

,Train,Test,% Train,% Test
BsmtFinSF1,0,1,0.0,0.0
BsmtFinSF2,0,1,0.0,0.0
BsmtUnfSF,0,1,0.0,0.0
TotalBsmtSF,0,1,0.0,0.0
BsmtFullBath,0,2,0.0,0.0
BsmtHalfBath,0,2,0.0,0.0
GarageCars,0,1,0.0,0.0
GarageArea,0,1,0.0,0.0


In [24]:
compare_columns(test, 'BsmtFinSF1', 'BsmtCond')

,BsmtFinSF1,BsmtCond
660,NaN,No BsmtCond


In [25]:
compare_columns(test, 'BsmtFinSF2', 'BsmtCond')

,BsmtFinSF2,BsmtCond
660,NaN,No BsmtCond


In [26]:
compare_columns(test, 'BsmtUnfSF', 'BsmtCond')

,BsmtUnfSF,BsmtCond
660,NaN,No BsmtCond


In [27]:
compare_columns(test, 'TotalBsmtSF', 'BsmtCond')

,TotalBsmtSF,BsmtCond
660,NaN,No BsmtCond


In [28]:
compare_columns(test, 'BsmtFullBath', 'BsmtCond')

,BsmtFullBath,BsmtCond
660,NaN,No BsmtCond
728,NaN,No BsmtCond


In [29]:
compare_columns(test, 'BsmtHalfBath', 'BsmtCond')

,BsmtHalfBath,BsmtCond
660,NaN,No BsmtCond
728,NaN,No BsmtCond


In [30]:
compare_columns(test, 'GarageCars', 'GarageCond')

,GarageCars,GarageCond
1116,NaN,No GarageCond


In [31]:
compare_columns(test, 'GarageArea', 'GarageCond')

,GarageArea,GarageCond
1116,NaN,No GarageCond


In [32]:
missing_numerical_columns_test = missing_values(
    train[numerical_features_lst[2:]], 
    test[numerical_features_lst[2:]]).query("Train > 0 or Test > 0")['Test'].index

for column in missing_numerical_columns_test:
    test[column] = test[column].apply(lambda area: 0.0 if pd.isna(area) else area)
    
del missing_numerical_columns_test, column

In [33]:
missing_values(train[numerical_features_lst[2:]], test[numerical_features_lst[2:]]).query("Train > 0 or Test > 0")

,Train,Test,% Train,% Test


In [34]:
test.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [35]:
# object_features_train = []
# float_features_train = []
# integer_features_train = []

# object_features_test = []
# float_features_test = []
# integer_features_test = []

# for column in train.columns:
#     if train[column].dtype == object:
#         object_features_train.append(column)
#     elif train[column].dtype == 'float64':
#         float_features_train.append(column)
#     elif train[column].dtype == 'int64':
#         integer_features_train.append(column)

# print(object_features_train, end='\n\n')
# print(float_features_train, end='\n\n')
# print(integer_features_train, end='\n\n')
# print(len(object_features_train) + len(float_features_train) + len(integer_features_train), end='\n\n')
# print(len(train.columns), end='\n\n')

# for column in test.columns:
#     if test[column].dtype == object:
#         object_features_test.append(column)
#     elif test[column].dtype == 'float64':
#         float_features_test.append(column)
#     elif test[column].dtype == 'int64':
#         integer_features_test.append(column)

# print(object_features_test, end='\n\n')
# print(float_features_test, end='\n\n')
# print(integer_features_test, end='\n\n')
# print(len(object_features_test) + len(float_features_test) + len(integer_features_test), end='\n\n')
# print(len(test.columns), end='\n\n')

In [36]:
# # Replace categorical missing values with the most frequent value
# print('\n***** Missing Categorical Features *****')
# for column in missing_categorical_features_test:
#     test[column] = test[column].apply(lambda value: test[column].mode()[0] if pd.isna(value) else value)
#     print(test[column].value_counts(), end='\n\n')
    
# # Replace numerical missing values with the mean 
# print('\n***** Missing Numerical Features *****')
# for column in missing_numerical_features_test:
#     test[column] = test[column].apply(lambda value: test[column].mean() if pd.isna(value) else value)

# # Replace categorical missing values with Not Available   
# print('\n***** No Categorical Features *****')
# for column in no_categorical_feautures_test:
#     test[column] = test[column].apply(lambda value: f'No {column}' if pd.isna(value) else value)
#     print(test[column].value_counts(), end='\n\n')

# # Replace numerical missing values with Not Available 
# print('\n***** No Numerical Features *****')
# for column in no_numerical_feautures_test:
#     test[column] = test[column].apply(lambda value: 0 if pd.isna(value) else value)

# # Replace None in MasVnrType column
# test.MasVnrType = test.MasVnrType.apply(lambda record: 'No MasVnrType' if record == 'None' else record)

In [37]:
# # Replace categorical missing values with the most frequent value
# print('\n***** Missing Categorical Features *****')
# for column in missing_categorical_features_train:
#     train[column] = train[column].apply(lambda value: train[column].mode()[0] if pd.isna(value) else value)
#     print(train[column].value_counts(), end='\n\n')
    
# # Replace numerical missing values with the mean 
# print('\n***** Missing Numerical Features *****')
# for column in missing_numerical_features_train:
#     train[column] = train[column].apply(lambda value: train[column].mean() if pd.isna(value) else value)

# # Replace categorical missing values with Not Available  
# print('\n***** No Categorical Features *****')
# for column in no_categorical_feautures_train:
#     train[column] = train[column].apply(lambda value: f'No {column}' if pd.isna(value) else value)
#     print(train[column].value_counts(), end='\n\n')

# # Replace numerical missing values with Not Available 
# print('\n***** No Numerical Features *****')
# for column in no_numerical_feautures_train:
#     train[column] = train[column].apply(lambda value: 0 if pd.isna(value) else value)
    
# # Replace None in MasVnrType column
# train.MasVnrType = train.MasVnrType.apply(lambda record: 'No MasVnrType' if record == 'None' else record)